In [47]:
import tensorflow as tf
import numpy as np

In [49]:
# Explicitly create a Graph object
tf.reset_default_graph()
graph = tf.Graph()

with graph.as_default():
    
    with tf.name_scope("variables"):
        # Variable to keep track of how many times the graph has been run
        global_step = tf.Variable(0, dtype=tf.int32, name="global_step")
        
        # Variable that keeps track of the sum of all output values over time:
        total_output = tf.Variable(0.0, dtype=tf.float32, name="total_output")
        #curr_value = tf.placeholder(tf.float32)
        alpha = tf.constant(0.05)
        prev_avg = tf.Variable(0.)
    
    # Primary transformation Operations
    with tf.name_scope("transformation"):
        
        # Separate input layer
        with tf.name_scope("input"):
            # Create input placeholder- takes in a Vector 
            a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")
    
        # Separate middle layer
        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_prod(a, name="product_b")
            c = tf.reduce_sum(a, name="sum_c")
            
    
        # Separate output layer
        with tf.name_scope("output"):
            output = tf.add(b, c, name="output")
            update_avg = alpha * a + (1 - alpha) * prev_avg
      
        
    with tf.name_scope("update"):
        # Increments the total_output Variable by the latest output
        update_total = total_output.assign_add(output)
        tf.assign(prev_avg, curr_avg)
        
        # Increments the above `global_step` Variable, should be run whenever the graph is run
        increment_step = global_step.assign_add(1)

        
        #Calculations
    
    # Summary Operations
    with tf.name_scope("summaries"):
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name="average")
        cumulative = tf.reduce_mean(a,name ="mean")
        # Creates summaries for output node
        tf.summary.scalar('Output', output)
        tf.summary.scalar('Sum_of_outputs_over_time', update_total) 
        tf.summary.scalar('Average_of_outputs_over_time', avg) 
        avg_hist = tf.summary.scalar("running_average", update_avg)
        value_hist = tf.summary.scalar("incoming_values", curr_value)

    
    # Global Variables and Operations
    with tf.name_scope("global_ops"):
        # Initialization Op
        init = tf.global_variables_initializer()    
        # Merge all summaries into one Operation
        merged_summaries = tf.summary.merge_all()

# Start a Session, using the explicitly created Graph
sess = tf.Session(graph=graph)

# Open a SummaryWriter to save summaries
writer = tf.summary.FileWriter('summaries1', graph)

# Initialize Variables
sess.run(init)


NameError: name 'curr_avg' is not defined

In [45]:
def run_graph(input_tensor):
    """
    Helper function; runs the graph with given input tensor and saves summaries
    """
    feed_dict = {a: input_tensor}

    summary_str, curr_avg = sess.run([merged_summaries, update_avg], feed_dict=feed_dict)
    
    sess.run(tf.assign(prev_avg, curr_avg))
    #print("Raw Data [i]:",i, " - Curr Average:",curr_avg)
    #writer.add_summary(summary_str)
    
    out, step, summary = sess.run([output, increment_step, merged_summaries], feed_dict=feed_dict)
    writer.add_summary(summary, global_step=step)

In [46]:
# Run the graph with various inputs
run_graph([2,8])
run_graph([3,1,3,3])
run_graph([8])
run_graph([1,2,3])
run_graph([11,4])
run_graph([4,1])
run_graph([7,3,1])
run_graph([6,3])
run_graph([0,2])
run_graph([4,5,6])

InvalidArgumentError: tags and values not the same shape: [] != [2] (tag 'summaries/running_average')
	 [[Node: summaries/running_average = ScalarSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](summaries/running_average/tags, transformation/intermediate_layer/add)]]

Caused by op 'summaries/running_average', defined at:
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477,

In [ ]:
# Write the summaries to disk
writer.flush()

In [ ]:
# Close the tf.summary.FileWriter
writer.close()

In [ ]:
# Close the session
sess.close()

To start TensorBoard after running this code, run the following command:

```
$ tensorboard --logdir 'summaries'
```

In [ ]:
tf=